In [ ]:
%run "../library.py"

In [2]:
from sklearn import preprocessing
from sklearn.lda import LDA
from sklearn import svm, cross_validation

In [3]:
# prepare full training and test dataset
# http://stats.stackexchange.com/questions/82923/mixing-continuous-and-binary-data-with-linear-svm
train_data = preprocessing.scale(np.hstack([train_cnn,train_bow,train_attribs]), axis=0)
# train_data = np.hstack([train_data,train_attribs])
test_data = np.hstack([test_cnn,test_bow,test_attribs])
# test_data = np.hstack([test_data,test_attribs])

means = np.mean(train_data,axis=0)
stds = np.std(train_data,axis=0)
test_data = (test_data - means[None,:])/ stds[None,:]

In [53]:
#svm classification
from sklearn import linear_model, svm
from sklearn.cross_validation import KFold

svm_accuracy = []
for train, test in  KFold(len(train_data),n_folds=3):
    X_train,Y_train, X_test,Y_test = train_data[train],train_labels[train],train_data[test],train_labels[test]
    score = svm.SVC(kernel='linear').fit(X_train,Y_train).score(X_test,Y_test)
    svm_accuracy.append(score)
    print score
print svm_accuracy, "mean: ", np.mean(svm_accuracy)

0.374
0.371
0.338
[0.374, 0.371, 0.33800000000000002] mean:  0.361


In [62]:
# LDA
sklearn_lda = LDA(n_components=200)
train_data_lda = sklearn_lda.fit_transform(train_data, train_labels)

/Users/2359media/anaconda/lib/python2.7/site-packages/sklearn/lda.py:371: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


In [86]:
svm_accuracy = []
for train, test in  KFold(len(train_data_lda),n_folds=5):
    X_train,Y_train, X_test,Y_test = train_data_lda[train],train_labels[train],train_data_lda[test],train_labels[test]
    score = svm.SVC(kernel='linear').fit(X_train,Y_train).score(X_test,Y_test)
    svm_accuracy.append(score)
    print score
    
print svm_accuracy, "mean: ", np.mean(svm_accuracy)

1.0
1.0
1.0
1.0
1.0
[1.0, 1.0, 1.0, 1.0, 1.0] mean:  1.0


In [80]:
#scale test set

test_data_lda = sklearn_lda.transform(test_data)
predict = svm.SVC(kernel='linear').fit(train_data_lda,train_labels).predict(test_data_lda)
kaggle_output(predict)

[['0000.jpg' 'soccer_field']
 ['0001.jpg' 'city']
 ['0002.jpg' 'recreation_room']
 ..., 
 ['0997.jpg' 'fastfood_restaurant']
 ['0998.jpg' 'dorm_room']
 ['0999.jpg' 'pasture']]


In [ ]:
# PCA + Adaboost
from sklearn.cross_validation import cross_val_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn import linear_model, svm
from sklearn.cross_validation import KFold


# train_data_pca = get_pca(train_data, 200)
# test_data_pca = get_pca(test_data, 200)
accuracies = []
for train, test in  KFold(len(train_data),n_folds=3):
    clf = AdaBoostClassifier(svm.SVC(probability=True,kernel='linear'),n_estimators=5)
    X_train,Y_train, X_test,Y_test = train_data[train],train_labels[train],train_data[test],train_labels[test]
    score = clf.fit(X_train,Y_train).score(X_test,Y_test)
    accuracies.append(score)
    print score
